<a href="https://colab.research.google.com/github/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes/blob/main/AML_AnomalySegmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!rm -r __MACOSX/

## Preparation

In [ ]:
# clone the github repo and move all files to the current working directory
!git clone https://github.com/AgneseRe/Real-Time-Anomaly-Segmentation-for-Road-Scenes.git
!mv Real-Time-Anomaly-Segmentation-for-Road-Scenes/* .
!rm -r Real-Time-Anomaly-Segmentation-for-Road-Scenes/

In [16]:
import os

In [17]:
# download required packages
!pip3 install --quiet cityscapesscripts
!pip3 install --quiet gdown
!pip3 install --quiet numpy
!pip3 install --quiet matplotlib
!pip3 install --quiet Pillow
!pip3 install --quiet torchvision
!pip3 install --quiet visdom
!pip3 install --quiet ood_metrics

In [18]:
# download and unzip validation dataset
!gdown 'https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta'
!file 'Validation_Dataset.zip'
!ls -lh Validation_Dataset.zip
!unzip -q 'Validation_Dataset.zip'
!mkdir validation_dataset && cp -pR Validation_Dataset/* validation_dataset/ && rm -R Validation_Dataset/
!unzip -l 'Validation_Dataset.zip'

Downloading...
From (original): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta
From (redirected): https://drive.google.com/uc?id=12YJq48XkCxQHjN3CmLc-zM5dThSak4Ta&confirm=t&uuid=627d4b85-46f7-4602-bf71-3d4f7a5c1fa1
To: /content/Validation_Dataset.zip
100% 329M/329M [00:02<00:00, 150MB/s]
Validation_Dataset.zip: Zip archive data, at least v2.0 to extract, compression method=store
-rw-r--r-- 1 root root 314M Apr 28  2024 Validation_Dataset.zip
Archive:  Validation_Dataset.zip
  Length      Date    Time    Name
---------  ---------- -----   ----
        0  2023-09-05 14:24   Validation_Dataset/
        0  2023-01-25 20:55   Validation_Dataset/RoadAnomaly21/
    10244  2023-09-05 14:24   Validation_Dataset/.DS_Store
      120  2023-09-05 14:24   __MACOSX/Validation_Dataset/._.DS_Store
        0  2023-02-16 07:46   Validation_Dataset/RoadAnomaly/
        0  2022-09-15 17:31   Validation_Dataset/RoadObsticle21/
        0  2022-09-16 14:14   Validation_Dataset/fs_static/
   

In [ ]:
# download datasets for training and testing
!csDownload leftImg8bit_trainvaltest.zip
!csDownload gtFine_trainvaltest.zip

# unzip the datasets
!unzip -q 'leftImg8bit_trainvaltest.zip' -d 'cityscapes'
!unzip -o -q 'gtFine_trainvaltest.zip' -d 'cityscapes'

# generate trainIds from labelIds
!CITYSCAPES_DATASET='cityscapes/' csCreateTrainIdLabelImgs

## Evaluation

### Step 2A - Compute AuPRC & FPR95TPR

In [19]:
%cd eval

/content/eval


In [ ]:
methods_list = ["msp", "maxlogit", "maxentropy"]
datasets_list = os.listdir("../validation_dataset")
for d in datasets_list:
  print(f"{d}")
  for m in methods_list:
    print(f"\t- {m}")
    input_path = f"../validation_dataset/{d}/images/*.*"
    !python evalAnomaly.py --input={input_path} --method={m}

RoadAnomaly21
	- msp
		AUPRC score: 29.100
		FPR@TPR95: 62.511
	- maxlogit
		AUPRC score: 38.320
		FPR@TPR95: 59.337
	- maxentropy
		AUPRC score: 31.005
		FPR@TPR95: 62.593
FS_LostFound_full
	- msp
		AUPRC score: 1.748
		FPR@TPR95: 50.763
	- maxlogit
		AUPRC score: 3.301
		FPR@TPR95: 45.495
	- maxentropy
		AUPRC score: 2.582
		FPR@TPR95: 50.368
fs_static
	- msp
		AUPRC score: 7.470
		FPR@TPR95: 41.823
	- maxlogit
		AUPRC score: 9.499
		FPR@TPR95: 40.300
	- maxentropy
		AUPRC score: 8.826
		FPR@TPR95: 41.523
RoadObsticle21
	- msp
		AUPRC score: 2.712
		FPR@TPR95: 64.974
	- maxlogit
		AUPRC score: 4.627
		FPR@TPR95: 48.443
	- maxentropy
		AUPRC score: 3.052
		FPR@TPR95: 65.600
RoadAnomaly
	- msp
		AUPRC score: 12.426
		FPR@TPR95: 82.492
	- maxlogit
		AUPRC score: 15.582
		FPR@TPR95: 73.248
	- maxentropy
		AUPRC score: 12.678
		FPR@TPR95: 82.632


### Step 2B - Compute AuPRC & FPR95TPR with temperature scaling

In [20]:
temperature_list = [0.5, 0.75, 1.0, 1.1, 1.2, 1.5, 2.0, 3.0, 5.0, 10.0, 50.0]
datasets_list = os.listdir("../validation_dataset")
for d in datasets_list:
  print(f"{d}")
  for t in temperature_list:
    print(f"\t- {t}")
    input_path = f"../validation_dataset/{d}/images/*.*"
    !python evalAnomaly.py --input={input_path} --method="msp" --temperature={t}

RoadAnomaly21
	- 0.5
		AUPRC score: 27.061
		FPR@TPR95: 62.731
	- 0.75
		AUPRC score: 28.156
		FPR@TPR95: 62.479
	- 1.0
		AUPRC score: 29.100
		FPR@TPR95: 62.511
	- 1.1
		AUPRC score: 29.410
		FPR@TPR95: 62.590
	- 1.2
		AUPRC score: 29.678
		FPR@TPR95: 62.724
	- 1.5
		AUPRC score: 30.258
		FPR@TPR95: 63.318
	- 2.0
		AUPRC score: 30.679
		FPR@TPR95: 64.721
	- 3.0
		AUPRC score: 30.674
		FPR@TPR95: 67.682
	- 5.0
		AUPRC score: 30.196
		FPR@TPR95: 71.594
	- 10.0
		AUPRC score: 29.526
		FPR@TPR95: 75.757
	- 50.0
		AUPRC score: 28.804
		FPR@TPR95: 80.014
FS_LostFound_full
	- 0.5
		AUPRC score: 1.280
		FPR@TPR95: 66.737
	- 0.75
		AUPRC score: 1.493
		FPR@TPR95: 51.848
	- 1.0
		AUPRC score: 1.748
		FPR@TPR95: 50.763
	- 1.1
		AUPRC score: 1.860
		FPR@TPR95: 50.387
	- 1.2
		AUPRC score: 1.972
		FPR@TPR95: 50.150
	- 1.5
		AUPRC score: 2.286
		FPR@TPR95: 49.456
	- 2.0
		AUPRC score: 2.677
		FPR@TPR95: 48.324
	- 3.0
		AUPRC score: 3.048
		FPR@TPR95: 46.893
	- 5.0
		AUPRC score: 3.252
		FPR@TPR95: 

### Step 3 - Train models with void classifier

### Step 4 - Project Extension